# mdx REST Client for Python 使用例

この Notebook では、mdx REST API にアクセスするためのPythonクライアントライブラリを使用し、mdx仮想マシンの作成と、仮想マシンに対する各種操作を行います。

## 準備

1. mdx REST API 認証トークン設定
2. mdx VM にSSHログインするためのキーペア作成
3. mdx の REST API エンドポイントに接続できることの確認

### mdx REST API 認証トークン設定

- mdx REST API 認証トークンは、[mdxユーザポータル](https://oprpl.mdx.jp/) の「トークン発行」により取得する。
  * 「トークン発行」は、ユーザポータル画面右上にあるユーザ名をクリックすると表示される。

In [ ]:
# mdx REST API 認証トークン設定
from getpass import getpass
mdx_token = getpass()

### mdx VM への SSH ログイン用キーペア作成

- mdx VM デプロイ時の設定項目に含まれる公開鍵を用意する。

In [ ]:
# ssh用キーペア作成
!mkdir -p -m 700 ~/.ssh
!test -f ~/.ssh/id_rsa || ssh-keygen -t rsa -f ~/.ssh/id_rsa -N ""

### mdx REST API エンドポイント接続確認

- HTTPステータスコード 200 が返ることを確認する。

In [ ]:
# デフォルトのresolverがIPv6のアドレスを返すことにより以降のAPIで接続不可の場合があるため、以下のコードを実行しておく。
def use_ipv4_only():
    import socket
    old_getaddrinfo = socket.getaddrinfo
    def new_getaddrinfo(*args, **kwargs):
        responses = old_getaddrinfo(*args, **kwargs)
        return [response
                for response in responses
                if response[0] == socket.AF_INET]
    socket.getaddrinfo = new_getaddrinfo

use_ipv4_only()

In [ ]:
!curl https://oprpl.mdx.jp -w '%{http_code}\n' -o /dev/null -s

## mdx 仮想マシンの作成

1. mdx REST Client for Python ライブラリの読み込み
2. mdx VM作成に必要なパラメータ設定
3. mdx VMデプロイ実行

### mdx REST Client for Python ライブラリの読み込み

In [ ]:
from mdx.mdx_ext import MdxResourceExt
mdx = MdxResourceExt(mdx_token)

### mdx VM作成に必要なパラメータ設定

- プロジェクトID
- ネットワークセグメントID
- sshログインのための公開鍵

自身が所属している（利用可能な）mdxのプロジェクト情報を確認する。

In [ ]:
import json
projects = mdx.get_assigned_projects()
print(json.dumps(projects[0]["projects"], indent=2))

以降のmdx操作対象とする「プロジェクト名」 (`name`) を設定する。

In [ ]:
project_name = ''
mdx.set_current_project_by_name(project_name)

操作対象として設定したプロジェクト情報を確認する。

In [ ]:
print(json.dumps(mdx.get_current_project(), indent=2))

プロジェクトで利用可能なットワークセグメントのリストを取得し、先頭のIDを設定する。

In [ ]:
segments = mdx.get_segments()
print(json.dumps(segments, indent=2))

segment_id = mdx.get_segments()[0]["uuid"]
print(segment_id)

sshログインのための公開鍵ファイルの内容を設定する

In [ ]:
import os
with open(os.path.expanduser('~/.ssh/id_rsa.pub')) as f:
    ssh_shared_key = f.read()
print(ssh_shared_key)

### mdx VMデプロイ

- ここでは、仮想マシンテンプレートとして「推奨版、東京大学制作、20250205-1029版」を使用する。
  * 利用可能な仮想マシンテンプレートの一覧は、 `get_vm_catalogs()` により確認できる。
- [ハードウェアのカスタマイズ項目](https://docs.mdx.jp/ja/main/create_vm.html#deploy-settings)は、以下の内容で設定する。
  * 仮想マシン名: `"rest-client-example_0001"` **（プロジェクト内で重複しないこと）**
  * パックタイプ: `"cpu"`
  * パック数: `3`
  * 仮想ディスク(GB): `40`
  * ストレージネットワーク: `"portgroup"`
  * サービスレベル (スポット仮想マシン `spot` または起動保証仮想マシン `guarantee`) : `"spot"`

VMのデプロイを実行する。デプロイ完了後、VM情報を出力する。

In [ ]:
DEFAULT_TEMPLATE_NAME = "UT-20250205-1029-ubuntu-2204-server"
DEFAULT_CATALOG = "32da6d82-bdca-405e-9209-62044bd92923"
vm_name = "rest-client-example_0001"

mdx_spec = dict(
    catalog=DEFAULT_CATALOG,
    template_name=DEFAULT_TEMPLATE_NAME,
    pack_num=3,
    pack_type="cpu",
    disk_size=40,
    gpu="0",
    network_adapters=[
        dict(
            adapter_number=1,
            segment=segment_id
        )
    ],
    shared_key=ssh_shared_key,
    storage_network="portgroup",
    service_level="spot",
)

info = mdx.deploy_vm(vm_name, mdx_spec)

print(json.dumps(info, indent=2))

VMに付与されたプライベートIPv4アドレス (先頭のアドレス) を取得する。

In [ ]:
host_ip_address = info["service_networks"][0]["ipv4_address"][0]
host_ip_address

### mdx VMへの疎通確認

In [ ]:
!ping -c 3 {host_ip_address}

### mdx VMへのSSHログイン確認

- Jupyter Notebook の Terminal を開き、VMに付与されたIPv4アドレス `host_ip_address` に対してSSH公開鍵認証で接続する。
- 仮想マシンテンプレート「推奨版、東京大学制作、20220412-2043版」におけるSSHログイン時の注意点は以下のとおり。
  * 初回ログイン時にはOSのパスワード設定が求められる
  * ログインユーザ名は `mdxuser`
  * 秘密鍵は、VMデプロイ時に指定した公開鍵 `ssh_shared_key` に対応する鍵ファイルを使用する

## mdx 仮想マシンの各種操作

### 仮想マシンの状態を取得する

In [ ]:
vm_info = mdx.get_vm_info(vm_name)
print(json.dumps(info, indent=2))

### プロジェクトの仮想マシンのリストを取得する

In [ ]:
lst = mdx.get_vm_list()
print(json.dumps(lst, indent=2))

### 仮想マシンをシャットダウンする

In [ ]:
mdx.power_shutdown_vm(vm_name, wait_for=True)

### 仮想マシンを削除する

In [ ]:
mdx.destroy_vm(vm_name, wait_for=True)

## （参考）仮想マシンテンプレート情報参照

- 利用可能な仮想マシンテンプレートのリストは以下のように取得することができる。
  - ユーザポータルでは REST API によるデプロイ時のパラメータとして必要な `catalog` (カタログID) および `template_name` (vCenter上の仮想マシンテンプレート名) を確認することができないため、 `get_vm_catalogs()` を用いて取得する。

In [ ]:
catalogs = mdx.get_vm_catalogs()
print(json.dumps(catalogs["results"], indent=2))